In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
from posarmctools.readdata import *
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, "../focalization_python")
from posarutils.other.PosarMCParameters import *

In [4]:
data_date = "2017_10_12_13_58_36"
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_12/" + data_date
params_filename = data_dir + "/" + data_date + "_parameters.xml"

In [5]:
params = PosarMCParameters( params_filename )

In [6]:
blocksPerFile = params.blocksPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

firstFile = 0
lastFile = 414
numberOfFiles = int( (lastFile - firstFile) / blocksPerFile + 1 )
#numberOfFiles = 60

threshold = 6000

print( "number of files = {}".format(numberOfFiles) )

number of files = 70


### Read the data

In [7]:
timeSerie_A = np.zeros( samplesPerFile )
timeSerie_B = np.zeros( samplesPerFile )

adc_A = np.zeros( (numberOfFiles, samplesPerFile) )
adc_B = np.zeros( (numberOfFiles, samplesPerFile) )

In [8]:
for k in range( numberOfFiles ):
    
    # read the data
    nb = str( int( k * blocksPerFile + firstFile) )
    filename = data_dir + "/record" + nb + ".bin"
    readFile( filename, samplesPerFile, timeSerie_A, timeSerie_B )
    
    print("k = {}, file = {} / {}".format(k, nb, numberOfFiles))
    
    adc_A[ k, : ] = timeSerie_A
    adc_B[ k, : ] = timeSerie_B

k = 0, file = 0 / 70
k = 1, file = 6 / 70
k = 2, file = 12 / 70
k = 3, file = 18 / 70
k = 4, file = 24 / 70
k = 5, file = 30 / 70
k = 6, file = 36 / 70
k = 7, file = 42 / 70
k = 8, file = 48 / 70
k = 9, file = 54 / 70
k = 10, file = 60 / 70
k = 11, file = 66 / 70
k = 12, file = 72 / 70
k = 13, file = 78 / 70
k = 14, file = 84 / 70
k = 15, file = 90 / 70
k = 16, file = 96 / 70
k = 17, file = 102 / 70
k = 18, file = 108 / 70
k = 19, file = 114 / 70
k = 20, file = 120 / 70
k = 21, file = 126 / 70
k = 22, file = 132 / 70
k = 23, file = 138 / 70
k = 24, file = 144 / 70
k = 25, file = 150 / 70
k = 26, file = 156 / 70
k = 27, file = 162 / 70
k = 28, file = 168 / 70
k = 29, file = 174 / 70
k = 30, file = 180 / 70
k = 31, file = 186 / 70
k = 32, file = 192 / 70
k = 33, file = 198 / 70
k = 34, file = 204 / 70
k = 35, file = 210 / 70
k = 36, file = 216 / 70
k = 37, file = 222 / 70
k = 38, file = 228 / 70
k = 39, file = 234 / 70
k = 40, file = 240 / 70
k = 41, file = 246 / 70
k = 42, file = 252 / 

In [9]:
A_reshaped = adc_A.reshape(numberOfFiles * rampsPerFile, samplesPerRamp)
B_reshaped = adc_B.reshape(numberOfFiles * rampsPerFile, samplesPerRamp)

In [10]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title( "first acquisition " + data_date )
plt.grid()

**Set the shifted flag properly**

In [11]:
shifted = 1

In [11]:
coupling_A = np.average(A_reshaped, 0)
#A_reshaped -= coupling_A

In [12]:
plt.figure()
plt.plot( coupling_A )
plt.title( "coupling " + data_date )
plt.grid()

In [16]:
ifft_a = np.fft.ifft( (A_reshaped[::10,3000:6000]), axis = 1 )

In [18]:
plt.matshow( 20 * np.log10( np.abs( ifft_a[:,50:500] ) ), aspect='auto' )
plt.title(data_date + " Track 1\nFiles {} to {}".format(firstFile, lastFile) )
plt.grid(color="w")

## Save the data

In [ ]:
np.save(data_dir + '/samplesA_track1_flightover1.npy', A_reshaped)

## Compute RD

In [12]:
import sys

In [13]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [14]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters( params_filename )

In [15]:
RDc = build_rd_from_data( params, A_reshaped, shifted )

shape of the samples matrix = (52500, 12000)
Data are shifted


In [16]:
coupling_RD_local = np.average(RDc, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}".format(firstFile, lastFile), coupling_RD_local )

In [19]:
RD2 = RDc - coupling_RD_local

In [20]:
ifft_RD = np.fft.ifft(RD2, axis = 1)

In [ ]:
nbPointsInRange = 300
x0 = 0
x1 = nbPointsInRange
y0 = 0
y1 = 0
extent = [ x0, x1, y0, y1 ]
rangeStart = 135
rangeStop = 250
azStart = 6000
azStop = 15000

In [22]:
plt.matshow( 20 * np.log10( np.abs( ifft_RD[::10, 0:500] ) ),
            aspect='auto')
plt.title(data_date)
ax = plt.gca()
#ax.axes.set_xlim(130,170)
#ax.axes.set_ylim(280,376)
plt.colorbar()